In [1]:
from langchain_unstructured import UnstructuredLoader
loader_local = UnstructuredLoader(
    file_path="nlp.pdf",
    strategy="hi_res",
)
docs_local = []
for doc in loader_local.lazy_load():
    docs_local.append(doc)

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: pikepdf C++ to Python logger bridge initialized
INFO: Reading PDF for file: nlp.pdf ...


In [2]:
len(docs_local)

313

In [3]:
print(docs_local[0].page_content)

1. What is Natural Language Processing (NLP)?


In [4]:
first_page_docs = [doc for doc in docs_local if doc.metadata.get("page_number") == 1]

for doc in first_page_docs:
    print(doc.page_content)

1. What is Natural Language Processing (NLP)?
Answer: Natural Language Processing (NLP) is a field of Artificial Intelligence (AI) concerned with the interactions between computers and human (natural) languages. It focuses on enabling computers to understand, interpret, and generate human language in a way that is both meaningful and useful.
2. Mention any two real-world applications of NLP.
Answer:
• Sentiment Analysis: Determining the emotional tone or attitude expressed in text, used for market research, brand monitoring, etc.
• Chatbots and Conversational AI: Building interactive agents that can engage in conversations with humans, provide customer service, answer questions, and more.
3. Define empirical laws in the context of NLP.
Zipf’s Law: When words are ranked according to their frequencies in a large enough collection of texts and then the frequency is plotted against the rank, the result is a logarithmic curve.
Heap's law states that the number of unique words V in a collect

In [5]:
section_data =[]
section =""
for docs in docs_local:
    if docs.metadata.get("category") == "Title":
        
        section_data.append(section)
        section =""
        section+= docs.page_content + "\n"
        
    else:
        section += docs.page_content + "\n"

       
   

In [6]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

# Initialize
model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

# Suppose section_data is a list of section texts
all_chunks = []
raw_chunks =[]

for i, section in enumerate(section_data):
    if len(section) > 0:
        chunks = text_splitter.split_text(section)
        print(f"Section {i} has {len(chunks)} chunks")
        for j, chunk in enumerate(chunks):
            # Optional: Add metadata like section number
            all_chunks.append(Document(
                page_content=chunk,
                metadata={"section_id": i, "chunk_id": j}
            ))
            raw_chunks.append(chunk)

# Create the vector index
embeddings = model.embed_documents(raw_chunks)

""" vectorindex = FAISS.from_documents(all_chunks, embeddings) """




INFO: Use pytorch device_name: cpu
INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Section 1 has 1 chunks
Section 2 has 1 chunks
Section 3 has 1 chunks
Section 4 has 1 chunks
Section 5 has 1 chunks
Section 6 has 1 chunks
Section 7 has 1 chunks
Section 8 has 6 chunks
Section 9 has 2 chunks
Section 10 has 1 chunks
Section 11 has 1 chunks
Section 12 has 3 chunks
Section 13 has 1 chunks
Section 14 has 1 chunks
Section 15 has 1 chunks
Section 16 has 2 chunks
Section 17 has 3 chunks
Section 18 has 1 chunks
Section 19 has 2 chunks
Section 20 has 1 chunks
Section 21 has 2 chunks
Section 22 has 1 chunks
Section 23 has 2 chunks
Section 24 has 1 chunks
Section 25 has 1 chunks
Section 26 has 5 chunks
Section 27 has 5 chunks
Section 28 has 4 chunks
Section 29 has 2 chunks
Section 30 has 1 chunks
Section 31 has 1 chunks
Section 32 has 1 chunks
Section 33 has 1 chunks
Section 34 has 3 chunks
Section 35 has 1 chunks
Section 36 has 1 chunks
Section 37 has 1 chunks
Section 38 has 1 chunks
Section 39 has 2 chunks
Section 40 has 2 chunks
Section 41 has 2 chunks
Section 42 has 3 chunks
S

' vectorindex = FAISS.from_documents(all_chunks, embeddings) '

In [7]:
from sklearn.cluster import KMeans
num_topics = 10
kmeans = KMeans(n_clusters=num_topics, random_state=42)
labels = kmeans.fit_predict(embeddings)


In [47]:
import os
import dotenv
dotenv.load_dotenv()
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.9,
    max_tokens=700,
    timeout=None,
    max_retries=2,
) 

In [ ]:
import re

cluster_topic_titles = {}
for cluster_id in set(labels):
    rep_idx = list(labels).index(cluster_id)
    rep_chunk = raw_chunks[rep_idx]

    # Updated prompt
    prompt = (
        f"Give a very short and clear title for the following topic content.\n"
        f"Just return the title. No explanations, no quotes, no alternatives, no extra text.\n\n"
        f"{rep_chunk}"
    )
    raw_title = llm.invoke(prompt).content.strip()

    # Sanitize: remove markdown, quotes, or prefixes
    clean_title = re.sub(r'^["“”‘’\'*]*|["“”‘’\'*.:]*$', '', raw_title)  # trim quotes, punctuation
    clean_title = re.sub(r'^(Topic Title|Title)\s*[:\-]\s*', '', clean_title, flags=re.IGNORECASE)
    clean_title = clean_title.split("\n")[0].strip()  # take only the first line

    cluster_topic_titles[cluster_id] = clean_title


In [20]:
import re
cluster_topic_titles = {}
for cluster_id in set(labels):
    rep_idx = list(labels).index(cluster_id)
    rep_chunk = raw_chunks[rep_idx]

    # Ask LLM to name this topic
    # Updated prompt
    prompt = (
        f"Give a very short and clear title for the following topic content.\n"
        f"Just return the title. No explanations, no quotes, no alternatives, no extra text.\n\n"
        f"{rep_chunk}"
    )
    raw_title = llm.invoke(prompt).content.strip()
    clean_title = re.sub(r'^["“”‘’\'*]*|["“”‘’\'*.:]*$', '', raw_title)  # trim quotes, punctuation
    clean_title = re.sub(r'^(Topic Title|Title)\s*[:\-]\s*', '', clean_title, flags=re.IGNORECASE)
    clean_title = clean_title.split("\n")[0].strip()
    cluster_topic_titles[cluster_id] = clean_title
    
labeled_chunks = []
for i, chunk_text in enumerate(raw_chunks):
    chunk_meta = {
        "section_id": all_chunks[i].metadata["section_id"],
        "chunk_id": all_chunks[i].metadata["chunk_id"],
        "cluster_id": int(labels[i]),
        "topic": cluster_topic_titles[labels[i]]
    }
    labeled_chunks.append({
        "text": chunk_text,
        "embedding": embeddings[i],
        "metadata": chunk_meta
    })

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [21]:
vectorstore = FAISS.from_texts(
    texts=[chunk["text"] for chunk in labeled_chunks],
    embedding=model,
    metadatas=[chunk["metadata"] for chunk in labeled_chunks]
)

# === Done! You can now use vectorstore.as_retriever() ===
retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))

In [13]:
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
import dotenv
dotenv.load_dotenv()

# Set your Groq API key


# Create the Groq LLM instance
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",  # or "llama2-70b-4096"
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Now create the QA chain



In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """You are a helpful assistant. 
Use ONLY the following context to answer the question. 
Do NOT use any prior knowledge. 
If the answer is not in the context, respond with "The answer is not available in the provided context."

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


In [13]:
query = "what is used to  capture the frequency of individul words in a document?"
result = qa_chain.invoke({"query": query})

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [14]:
print("Answer:", result['result'])

Answer: Tokenization. 

Tokenization breaks down text into individual words or phrases, known as tokens. This process captures the frequency of individual words in a document.


In [15]:
for doc in result["source_documents"]:
    print("Chunk:", doc.page_content)
    print("Section ID:", doc.metadata.get("section_id"))
    print("Chunk ID:", doc.metadata.get("chunk_id"))


Chunk: Example:
Consider a corpus such as a collection of news articles. In many English texts, the word “the” is the most frequent. Suppose “the” occurs 10,000 times; then Zipf’s Law suggests that the second most common word might occur roughly 5,000 times, the third about 3,300 times, and so on. Although real data rarely follow the law perfectly (especially at the very high and low frequency ends), the overall pattern is striking. This regularity has been observed across languages and types of text .
Section ID: 40
Chunk ID: 0
Chunk: Tokenization: This breaks down text into individual words or phrases, known as tokens. This is often the first step in text processing.
Section ID: 27
Chunk ID: 4
Chunk: 17. Describe the process of text pre-processing with suitable examples.
Text cleansing Remove faumbers. symbols, marks Creatirg Document Stemming = Keyword Matrix ioKm) Creating 3 corpus Tokenization Removing stop words
Text preprocessing typically involves the following steps:
• Lowerca

In [45]:
print(result["source_documents"])

[Document(id='a8d5a770-c976-484a-99a9-20a50b086c15', metadata={'section_id': 40, 'chunk_id': 0, 'cluster_id': 5, 'topic': '"Empirical Laws in NLP"'}, page_content='Example:\nConsider a corpus such as a collection of news articles. In many English texts, the word “the” is the most frequent. Suppose “the” occurs 10,000 times; then Zipf’s Law suggests that the second most common word might occur roughly 5,000 times, the third about 3,300 times, and so on. Although real data rarely follow the law perfectly (especially at the very high and low frequency ends), the overall pattern is striking. This regularity has been observed across languages and types of text .'), Document(id='7fad23d3-7d3d-4511-8eda-a0e7558298d2', metadata={'section_id': 27, 'chunk_id': 4, 'cluster_id': 0, 'topic': '"Introduction to Natural Language Processing (NLP)"'}, page_content='Tokenization: This breaks down text into individual words or phrases, known as tokens. This is often the first step in text processing.'), D

In [22]:
# Get all stored documents from FAISS
all_docs = vectorstore.similarity_search("placeholder", k=len(vectorstore.docstore._dict))

# Extract and print all unique topics
topics = set()
for doc in all_docs:
    topic = doc.metadata.get("topic")
    if topic:
        topics.add(topic)


print("Unique Topics:")
for topic in sorted(topics):
    print("-", topic)


Unique Topics:
- Data Retrieval
- Definition
- Empirical Laws in NLP
- Finite-State Methods in Morphology
- Minimum Character Edits
- Natural Language Processing (NLP)
- Real-World Applications of NLP
- Role of Smoothing in Language Models
- Text Preprocessing
- Types of Ambiguity in NLP


In [23]:
def get_chunks_by_topic(vectorstore, topic_query):
    all_docs = vectorstore.similarity_search("placeholder", k=len(vectorstore.docstore._dict))
    
    topic_chunks = []
    for doc in all_docs:
        if doc.metadata.get("topic", "").lower() == topic_query.lower():
            topic_chunks.append(doc.page_content)
    
    return topic_chunks


In [24]:
topic = "Data Retrieval"  # or input("Enter topic: ")
chunks = get_chunks_by_topic(vectorstore, topic)

print(f"\nFound {len(chunks)} chunks for topic '{topic}':\n")
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}:\n{chunk}\n")



Found 18 chunks for topic 'Data Retrieval':

Chunk 1:
Retrieval

Chunk 2:
iii. N-gram models struggle to capture longer-distance context clues. It has been shown that after 6-grams, the gain in performance is limited. This is unfavorable in tasks where that is a particularly desirable feature and necessity.

Chunk 3:
3.Handling data sparsity: Language models often encounter rare or infrequent n-grams that have limited or no training examples. Smoothing techniques estimate probabilities for these unseen events by redistributing probability mass from observed events. This helps in more accurately modeling the likelihood of unseen or infrequent n-grams.

Chunk 4:
speech recognition, text prediction

Chunk 5:
9. Write the formula for calculating bigram probabilities.
In a bigram model, the probability of a word wi given its preceding word wi−1 is calculated using the maximum likelihood estimate (MLE):
Here:
P(w; | w;1) — count(wj_1, w;) count (w;_1)
• count
(wi−1,wi) is the number of time

In [ ]:
template =  """
You are an expert educational content designer.

Your task is to transform the following educational content about **"{topic}"** into 1–2 highly informative visuals with accompanying audio narration.

The text chunks contain various explanations. Read through all the chunks and extract key concepts.

Constraints:
- You may generate a maximum of 2 images but try making one only if is needed two.
- Choose the best way to visually convey each concept. (e.g., flowcharts, word trees, graphs, scene illustrations).
- Keep it clear and accurate, suitable for middle school to college-level learners.

For each image:
1. Give a detailed prompt for image generation.
2. Write a short audio narration (4–6 sentences) explaining the image.
3. Ensure the audio is clear and concise, suitable for educational purposes and refers to the visual elements ,use words like "In this image..." or "As shown in the image...".

Use this format strictly:
{{
  "image1": {{
    "descriptor": "<visual description prompt for image generation>",
    "speech": "<audio narration explaining the visual>"
  }},
  "image2": {{
    "descriptor": "<visual description prompt for second image>",
    "speech": "<audio narration explaining the second visual>"
  }}
}}

Content Chunks:
{chunks}
"""
prompt = PromptTemplate.from_template(template)


In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
topic = "Data Retrieval"  # or input("Enter topic: ")
chunks = get_chunks_by_topic(vectorstore, topic)
parser = StrOutputParser()

# Chain
chain = prompt | llm | parser

# Call chain with topic and chunks
response = chain.invoke({
    "topic": topic,
    "chunks": "\n".join(chunks)
})


INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [49]:
print(response)

Here are two highly informative visuals along with accompanying audio narrations to explain the concept of "Data Retrieval" using N-gram models:

{
  "image1": {
    "descriptor": "A diagram that visually breaks down the N-gram Model. The model has a central circle labeled 'Input Word'. Radiating from this circle are arrows representing the preceding N-1 words. The arrows lead to a series of connected circles labeled with different N-gram sizes (e.g., Bigram, Trigram, N-gram). Each circle represents a word. Connections between the circles demonstrate how the N-gram model predicts the likelihood of a word based on the preceding N-1 words. A smaller circle with 'Probability' written above it is connected to the central circle.",
    "speech": "In this image, we see an N-gram model. It starts with an input word, and from there, it uses the preceding N-1 words to predict the likelihood of the next word. The larger the N-gram size, the more context it considers. As shown in the image, the l

In [ ]:
import json
import re
import requests
from gtts import gTTS


match = re.search(r"\{\s*\"image1\".*\}", response, re.DOTALL)

if match:
    json_str = match.group(0)
    parsed = json.loads(json_str)

    for i, (key, value) in enumerate(parsed.items(), 1):
        descriptor = value["descriptor"]
        speech_text = value["speech"]

        # -------- Stable Diffusion (Replicate API) --------
        replicate_api_token = "your_replicate_api_key"  # Replace with your API key
        headers = {"Authorization": f"Token {replicate_api_token}"}

        data = {
            "version": "your-model-version-id",  # Insert correct version ID for SD
            "input": {
                "prompt": f"Generate a clean, educational visual: {descriptor}"
            }
        }

        response = requests.post(
            "https://api.replicate.com/v1/predictions",
            headers=headers,
            json=data
        )

        prediction = response.json()
        image_url = prediction["urls"]["get"]

        # Polling the endpoint until the image is ready (optional step here)
        # You might need to implement polling if image generation is async.

        # Download the image
        img_response = requests.get(image_url)
        with open(f"image{i}.png", "wb") as f:
            f.write(img_response.content)

        # -------- Audio Generation (gTTS) --------
        tts = gTTS(speech_text)
        tts.save(f"image{i}.mp3")

        print(f"✅ Saved image{i}.png and image{i}.mp3")
else:
    print("❌ JSON not found in response.")


<re.Match object; span=(146, 2330), match='{\n  "image1": {\n    "descriptor": "A diagram th>
image1:
  Descriptor: A diagram that visually breaks down the N-gram Model. The model has a central circle labeled 'Input Word'. Radiating from this circle are arrows representing the preceding N-1 words. The arrows lead to a series of connected circles labeled with different N-gram sizes (e.g., Bigram, Trigram, N-gram). Each circle represents a word. Connections between the circles demonstrate how the N-gram model predicts the likelihood of a word based on the preceding N-1 words. A smaller circle with 'Probability' written above it is connected to the central circle.
  Speech: In this image, we see an N-gram model. It starts with an input word, and from there, it uses the preceding N-1 words to predict the likelihood of the next word. The larger the N-gram size, the more context it considers. As shown in the image, the larger N-gram models like N-grams have a more complex structure with mult